# Gatsiva Sample Analysis
## Visualizing Return Results

This notebook shows how to visualize the distribution of returns after an event occurs utilizing the Gatsiva API.

API access is **currently limited to beta testers and collaborators**. For more information on how to utilize the Gatsiva API or to request access and an API key, please visit the [Gatsiva Website](https://gatsiva.com). For more information on the Gatsiva API, please visit the [Gatsiva API Documentation](https://gatsiva.com/docs).

## Enter Input Information Here

In [ ]:
symbol = 'BTC:USD:hourly'
condition = 'close(1) < low(168) 1 period ago'
stddev_bands = 1.5
periods = 336
usingdatafrom = "2016-12-04"
usingdatato = "2018-01-10"
apikey = "<your_api_key>"

#### Python code to get the results - see results further below

In [ ]:
# Install Prerequesities if needed
#
# Run this (uncomment below) to install / upgrade Plotly
#!pip install plotly --upgrade


In [ ]:
# Import utility libraries we will need
import requests
import json
import numpy as np
import pandas as pd
import warnings

# Import plotly for graphing awesomeness
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

# To allow the jupyter notebook to render in offline mode
init_notebook_mode(connected=True)

# Ignore warnings
warnings.filterwarnings('ignore')

# Form the request
request_data = {'symbol':symbol,'condition_id':1,'condition':condition,"periods":periods,'from':usingdatafrom,'to':usingdatato}
bearer_token = 'Bearer ' + apikey
my_headers = {'Accept': 'application/json', 'Authorization': bearer_token}

# Send the request
analytic = requests.post('https://api.gatsiva.com/api/v1/conditions/return_profile',json=request_data,headers=my_headers)

# Pull out the results and format it into a pandas data frame
results = analytic.json()['results']
df = pd.read_json(json.dumps(results), orient='records')

# Now add two new columns to support lines for 2x stddevs
df['upper'] = df['mean'] + ((stddev_bands/2) * df['stddev'])
df['lower'] = df['mean'] - ((stddev_bands/2) * df['stddev'])

# Get the number of observations
num_observations = results[0]['n']

# Create the data structure needed for plotly
data = [
    Scatter(
        x=df['returnperiod'],
        y=df['mean'],
        name='mean'
    ),
    Scatter(
        x=df['returnperiod'], 
        y=df['upper'],
        name=str(stddev_bands) + ' stddev band upper'
    ),
    Scatter(
        x=df['returnperiod'], 
        y=df['lower'],
        name=str(stddev_bands) + ' stddev band lower'

    )
]

# Create the layout needed for plotly
layout = Layout(
    title=condition + ' -- ' + str(num_observations) + ' observations',
    yaxis=dict(title='return'),
    xaxis=dict(title='periods after event'),
    legend=dict(orientation="h",x=0.15, y=-0.2)
)

# Create the figure
fig = Figure(data=data, layout=layout)

## Results

In [ ]:
# Plot the result
iplot(fig)

### Detailed Results

Below here, you can see the detailed results within the dataframe. Note how the upper and lower columns were added in the earlier step.

In [ ]:
df